# Part 1 - Data Preprocessing


In [0]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [0]:
# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

In [0]:
# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
ct = ColumnTransformer([("Geography", OneHotEncoder(), [1])],    remainder = 'passthrough')
X = ct.fit_transform(X)
X = X[:, 1:]
X = np.array(X)

In [0]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [0]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Part 2 - Making the architecture of the ANN

In [7]:
# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

Using TensorFlow backend.


In [8]:
# Initialising the ANN
classifier = Sequential()

In [9]:
# Adding the input layer and the first hidden layer with dropout
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
classifier.add(Dropout(rate = 0.1))




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dropout(rate = 0.1))

In [0]:
# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

In [12]:
# Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [13]:
# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)




Epoch 1/100





8000/8000 [==============================] - 2s 214us/step - loss: 0.4868 - acc: 0.7959
Epoch 2/100
8000/8000 [==============================] - 1s 132us/step - loss: 0.4372 - acc: 0.7960
Epoch 3/100
8000/8000 [==============================] - 1s 132us/step - loss: 0.4313 - acc: 0.7960
Epoch 4/100
8000/8000 [==============================] - 1s 130us/step - loss: 0.4310 - acc: 0.7960
Epoch 5/100
8000/8000 [==============================] - 1s 128us/step - loss: 0.4292 - acc: 0.7960
Epoch 6/100
8000/8000 [==============================] - 1s 127us/step - loss: 0.4278 - acc: 0.7960
Epoch 7/100
8000/8000 [==============================] - 1s 130us/step - loss: 0.4286 - acc: 0.8185
Epoch 8/100
8000/8000 [==============================] - 1s 131us/step - loss: 0.4273 - acc: 0.8210
Epoch 9/100
8000/8000 [==============================] - 1s 129us/step - loss: 0.4271 - acc: 0.8235
Epoch 10/100
8000/8000 [==============================] - 1s 128us/step - loss: 0.4275 - acc

# Part 3 - Making predictions and evaluating the model

We are going to predict if the customer with the following informations will leave the bank:

* Geography: France
* Credit Score: 600
* Gender: Male
* Age: 40
* Tenure: 3
* Balance: 60000
* Number of Products: 2
* Has Credit Card: Yes
* Is Active Member: Yes
* Estimated Salary: 50000

In [14]:
new_prediction = classifier.predict(sc.transform(np.array([[0.0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])))
new_prediction = (new_prediction > 0.5)
new_prediction

array([[False]])

In [0]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [16]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[1556,   39],
       [ 278,  127]])

In [17]:
testing_accuracy = (131 + 1552)/2000
print(testing_accuracy)

0.8415


# Part 4 - Evaluating and Tuning the ANN


In [0]:
# Evaluating the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier, batch_size = 10, epochs = 100)
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs = -1)
mean = accuracies.mean()
var = accuracies.std()

In [19]:
print(mean)
print(var)

0.8397499948367477
0.010074720983459141


In [20]:
# Tuning the ANN
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
    classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
    classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier)
parameters = {'batch_size'  : [25, 32],
              'epochs' : [100, 500],
              'optimizer' :  ['adam', 'rmsprop']}
grid_search = GridSearchCV(estimator = classifier, 
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 5,
                           n_jobs = -1)
grid_search = grid_search.fit(X_train, y_train)
best_parameters =  grid_search.best_params_
best_accuracy = grid_search.best_score_

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Epoch 1/500
8000/8000 [==============================] - 1s 75us/step - loss: 0.5973 - acc: 0.7944
Epoch 2/500
8000/8000 [==============================] - 0s 48us/step - loss: 0.4463 - acc: 0.7960
Epoch 3/500
8000/8000 [==============================] - 0s 48us/step - loss: 0.4267 - acc: 0.7960
Epoch 4/500
8000/8000 [==============================] - 0s 49us/step - loss: 0.4180 - acc: 0.8071
Epoch 5/500
8000/8000 [==============================] - 0s 52us/step - loss: 0.4105 - acc: 0.8259
Epoch 6/500
8000/8000 [==============================] - 0s 47us/step - loss: 0.4026 - acc: 0.8276
Epoch 7/500
8000/8000 [==============================] - 0s 46us/step - loss: 0.3961 - acc: 0.8294
Epoch 8/500
8000/8000 [==============================] - 0s 48us/step - loss: 0.3903 - acc: 0.8302
Epoch 9/500
8000/8000 [==============================] - 0s 46us/step - loss: 0.3855 - acc: 0.8319
Epoch 10/500
8000/8000 [==============================] - 0s 49us/step - loss: 0.3812 - acc: 0.8325
Epoch 11/

In [23]:
print(best_parameters)
print(best_accuracy)

{'batch_size': 25, 'epochs': 500, 'optimizer': 'rmsprop'}
0.8487500000000001


So the optimal neural network would be with batch size 25, 500 epochs and the rmsprop as optimizer. This model has an 84.87% accuracy in comparison to our previous model which had 83.97%.